In [1]:
import pandas as pd


In [2]:
df = pd.read_csv('VA_incrementals1_test.csv')
df_binary = df['FULL_ADDRESS']

#Taking only the selected two attributes from the dataset
df_binary.columns = ['FULL_ADDRESS']
#display the first 5 rows
#df_binary.head()


In [3]:
def extract_street_addresses(dataframe, address_column):
    street_addresses = []  # Initialize an empty list to store extracted street addresses

    #Traverse the rows of the dataframe
    for index, row in dataframe.iterrows():
        full_address = row['FULL_ADDRESS'] 

        #Extract the desired part of the address using string manipulation
        #Split the full address by comma and take the first part as the street address
        street_address = full_address.split(',')[0].strip()

        #Add the street address to the list
        street_addresses.append(street_address)

    #Create a new dataframe with the extracted street addresses
    extracted_addresses_df = pd.DataFrame({
        'Extracted Street Addresses': street_addresses,
        'Incremental': 1
    })
    return extracted_addresses_df

In [4]:
street_address_list = extract_street_addresses(df, 'FULL_ADDRESS')

print("Extracted Street Addresses:")
for address in street_address_list:
    print(street_address_list)

Extracted Street Addresses:
     Extracted Street Addresses  Incremental
0                     0 COAL RD            1
1                 0 NEW SERVICE            1
2                 0 NEW SERVICE            1
3                 0 NEW SERVICE            1
4               0 PATTERSON TRL            1
...                         ...          ...
4994           1039 PRESTON AVE            1
4995      1039 S WINCHESTER AVE            1
4996               1039 TORI CT            1
4997               1039 TORI CT            1
4998             1039 TUPELO CT            1

[4999 rows x 2 columns]
     Extracted Street Addresses  Incremental
0                     0 COAL RD            1
1                 0 NEW SERVICE            1
2                 0 NEW SERVICE            1
3                 0 NEW SERVICE            1
4               0 PATTERSON TRL            1
...                         ...          ...
4994           1039 PRESTON AVE            1
4995      1039 S WINCHESTER AVE            1
49

In [5]:
def mark_adjacent_duplicates(dataframe):
   #Get a list of unique street addresses from the dataframe
    
    unique_addresses_list = dataframe['Extracted Street Addresses'].drop_duplicates().tolist()

    #Iterate through the unique street addresses and update boolean values
    for address in unique_addresses_list:
        address_rows = dataframe.index[dataframe['Extracted Street Addresses'] == address]

        #Check for adjacent rows with the same address and update boolean values accordingly
        for row_idx in address_rows:
            prev_row_idx = row_idx - 1
            next_row_idx = row_idx + 1

            if prev_row_idx in dataframe.index and dataframe.loc[prev_row_idx, 'Extracted Street Addresses'] == address:
                dataframe.loc[row_idx, 'Incremental'] = 0
            elif next_row_idx in dataframe.index and dataframe.loc[next_row_idx, 'Extracted Street Addresses'] == address:
                dataframe.loc[row_idx, 'Incremental'] = 0

    return dataframe


In [6]:
street_address_list = mark_adjacent_duplicates(street_address_list)
print(street_address_list)

     Extracted Street Addresses  Incremental
0                     0 COAL RD            1
1                 0 NEW SERVICE            0
2                 0 NEW SERVICE            0
3                 0 NEW SERVICE            0
4               0 PATTERSON TRL            1
...                         ...          ...
4994           1039 PRESTON AVE            1
4995      1039 S WINCHESTER AVE            1
4996               1039 TORI CT            0
4997               1039 TORI CT            0
4998             1039 TUPELO CT            1

[4999 rows x 2 columns]


In [7]:
street_address_list.drop('Extracted Street Addresses', axis=1, inplace=True)
print(street_address_list)

      Incremental
0               1
1               0
2               0
3               0
4               1
...           ...
4994            1
4995            1
4996            0
4997            0
4998            1

[4999 rows x 1 columns]


In [8]:
df.drop('Incremental', axis = 1, inplace = True)
merged_df = df.join(street_address_list)
print(merged_df)


        FULLLOCHID                                     FULL_ADDRESS  \
0     7.970000e+18  0 COAL RD, STUARTS DRAFT, VA 24477 COAL RD CELL   
1     8.210000e+18              0 NEW SERVICE, EAGLE ROCK, VA 24085   
2    -3.290000e+18               0 NEW SERVICE, FINCASTLE, VA 24090   
3     3.330000e+18              0 NEW SERVICE, TROUTVILLE, VA 24175   
4    -2.000000e+17            0 PATTERSON TRL, EAGLE ROCK, VA 24085   
...            ...                                              ...   
4994  8.370000e+18      1039 PRESTON AVE, CHARLOTTESVILLE, VA 22903   
4995 -6.740000e+18      1039 S WINCHESTER AVE, WAYNESBORO, VA 22980   
4996 -5.270000e+18                   1039 TORI CT, FOREST, VA 24551   
4997 -2.970000e+18                1039 TORI CT, LYNCHBURG, VA 24503   
4998  7.530000e+18        1039 TUPELO CT, CHARLOTTESVILLE, VA 22903   

            LOCHID                                  LOC_ADDRESS  \
0     6.940000e+18           0 COAL RD, STUARTS DRAFT, VA 24477   
1     8.21000

In [10]:
#Save DataFrame to CSV file

df.to_csv('VA_incrementals_test.csv', index=False)
